## Python notebook for generating the STAC catalog json and corresponding Item json for Raster & Vector layers

### Tools:
1. Pystac 
2. Rasterio
3. Geopandas
4. Matplotlib

This notebook returns Catalog json for Raster and Vector layers.

### 1. Importing the required modules

In [1]:
import os
import json
import xml.etree.ElementTree as ET
from datetime import datetime

import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import pystac
import sys
import constants
from shapely.geometry import mapping, box





/home/vishnu/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### 2. Defining the variables used in the notebook

In [2]:

base_dir="data/"
qml_path="data/style_file.qml"
vector_qml_file ="data/swb_style.qml"

raster_filename="saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif"
vector_filename="swb2_saraikela-kharsawan_gobindpur.geojson"

corestack_dir = os.path.join(base_dir, "CorestackCatalogs")
gobindpur_dir = os.path.join(corestack_dir, "gobindpur")
raster_dir = os.path.join(gobindpur_dir, "raster")
vector_dir = os.path.join(gobindpur_dir, "vector")

os.makedirs(raster_dir, exist_ok=True)
os.makedirs(vector_dir, exist_ok=True)


raster_path = os.path.join(base_dir, raster_filename)
vector_path = os.path.join(base_dir, vector_filename)

raster_thumbnail = os.path.join(raster_dir, "raster_thumbnail.png")
vector_thumbnail = os.path.join(vector_dir, "vector_thumbnail.png")

raster_style_file = os.path.join(base_dir, "style_file.qml")
vector_style_file = os.path.join(base_dir, "swb_style.qml")


### 3. For Raster layers the data range fecthed from filename

In [3]:
def extract_raster_dates_from_filename(filename):
    try:
        print(filename)
        parts = filename.split('_')
        start_date = datetime.strptime(parts[2], "%Y-%m-%d")
        end_date = datetime.strptime(parts[3], "%Y-%m-%d")
        print(start_date)
        print(end_date)
    except Exception as e:
        raise ValueError(f"Failed to extract raster dates from filename '{filename}': {e}")
        
    return start_date, end_date    

In [4]:
extract_raster_dates_from_filename(filename=raster_filename)

saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif
2023-07-01 00:00:00
2024-06-30 00:00:00


(datetime.datetime(2023, 7, 1, 0, 0), datetime.datetime(2024, 6, 30, 0, 0))

### 4. Parsing the QML file for Raster Layers

In [5]:


def parse_qml_classes(qml_path):
    tree = ET.parse(qml_path)
    root = tree.getroot()
    classes = []

    for entry in root.findall(".//paletteEntry"):
        class_info = {}
        for attr_key, attr_value in entry.attrib.items():
            if attr_key == "value":
                try:
                    class_info[attr_key] = int(attr_value)
                except ValueError:
                    class_info[attr_key] = attr_value
            else:
                class_info[attr_key] = attr_value
        classes.append(class_info)
    return classes

### 5. Generating the thumbnails from the files 

In [6]:
def generate_raster_thumbnail(tif_path, out_path):
    with rasterio.open(tif_path) as src:
        arr = src.read(1)
    plt.figure(figsize=(3, 3))
    plt.imshow(arr, cmap="tab20")
    plt.axis('off')
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def generate_vector_thumbnail(vector_path, out_path):
    gdf = gpd.read_file(vector_path)
    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
    fig, ax = plt.subplots(figsize=(3, 3))
    fig.patch.set_facecolor("white")
    ax.set_facecolor("white")
    gdf.plot(ax=ax, color="lightblue", edgecolor="blue", linewidth=0.5)
    ax.axis('off')
    plt.savefig(out_path, dpi=150, bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor())
    plt.close()


### 6. Creating the Raster items and adding the assets

In [7]:
def create_raster_item(filename):
    try:
        start_date, end_date = extract_raster_dates_from_filename(filename=raster_filename)
    except ValueError as e:
        raise RuntimeError(f"Raster item creation failed")
    
    with rasterio.open(raster_path) as src:
        bounds = src.bounds
        geom = mapping(box(*bounds))
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]

    generate_raster_thumbnail(raster_path, raster_thumbnail)
    style_info = parse_qml_classes(raster_style_file)

    print(style_info)
   
    style_json_path = os.path.join(raster_dir, "legend.json")
    with open(style_json_path, "w") as f:
        json.dump(style_info, f, indent=2)

    

    

    item = pystac.Item(
        id=constants.raster_lulc_id,
        geometry=geom,
        bbox=bbox,
        datetime=start_date,
        start_datetime= start_date,
        end_datetime= end_date,
        properties={
            "title" :constants.raster_lulc_title,
            "description":constants.raster_lulc_description,
            "lulc:classes": style_info,
            
        }
    )
    print(item)
    

    item.add_asset("data", pystac.Asset(
        href=f"{constants.data_url}/{raster_filename}",
        media_type=pystac.MediaType.GEOTIFF,
        roles=["data"],
        title="Raster Layer"
    ))
    item.add_asset("thumbnail", pystac.Asset(
        href=f"{constants.base_url}/raster/thumbnail.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Raster Thumbnail"
    ))
    item.add_asset("legend", pystac.Asset(
        href=f"{constants.base_url}/raster/legend.json",
        media_type=pystac.MediaType.JSON,
        roles=["metadata"],
        title="Legend JSON"
    ))
    item.add_asset("style", pystac.Asset(
        href=f"{constants.data_url}/../{qml_path}",
        media_type=pystac.MediaType.XML,
        roles=["metadata"],
        title="Raster style"
    ))

    item.set_self_href(os.path.join(raster_dir, "item.json"))
    item.save_object()
    return item


### 7.Creating the Vector items and adding the assets

In [8]:
def create_vector_item(vector_filename):
    start_date = constants.DEFAULT_START_DATE
    end_date = constants.DEFAULT_END_DATE

    gdf = gpd.read_file(vector_path)
    geom = mapping(gdf.union_all())
    bounds = gdf.total_bounds
    bbox = [float(b) for b in bounds]

    generate_vector_thumbnail(vector_path, vector_thumbnail)
    
    
    

    vector_metadata = {
        "number_of_records": gdf.shape[0], 
        "column_types":{col: str(dtype) for col, dtype in gdf.dtypes.items()}
    }
    
    vector_metadata_final = json.dumps(vector_metadata, indent=4)




    item = pystac.Item(
        id=constants.swb_vector_id,
        geometry=geom,
        bbox=bbox,
        datetime=start_date,
        start_datetime= start_date,
        end_datetime= end_date,
        properties={
            "title": constants.swb_vector_title,
            "description": constants.swb_vector_description,
            "vector_metadata": vector_metadata_final,
        }
    )

    print(vector_metadata_final)

    item.add_asset("data", pystac.Asset(
        href=f"{constants.data_url}/{vector_filename}",
        media_type=pystac.MediaType.GEOJSON,
        roles=["data"],
        title="Vector Layer"
    ))
    item.add_asset("thumbnail", pystac.Asset(
        href=f"{constants.base_url}/vector/thumbnail.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Vector Thumbnail"
    ))
    item.add_asset("style", pystac.Asset(
        href=f"{constants.data_url}/../{vector_style_file}",
        media_type=pystac.MediaType.XML,
        roles=["style"],
        title="Vector style"
    ))
    
    item.set_self_href(os.path.join(vector_dir, "item.json"))
    item.save_object()
    return item


In [9]:
vector_item=create_vector_item(vector_filename)
print(vector_item)


{
    "number_of_records": 2306,
    "column_types": {
        "id": "object",
        "MWS_UID": "object",
        "UID": "object",
        "any": "int32",
        "area_17-18": "float64",
        "area_18-19": "float64",
        "area_19-20": "float64",
        "area_20-21": "float64",
        "area_21-22": "float64",
        "area_22-23": "float64",
        "area_23-24": "float64",
        "area_ored": "float64",
        "category_sq_m": "object",
        "k_17-18": "float64",
        "k_18-19": "float64",
        "k_19-20": "float64",
        "k_20-21": "float64",
        "k_21-22": "float64",
        "k_22-23": "float64",
        "k_23-24": "float64",
        "kr_17-18": "float64",
        "kr_18-19": "float64",
        "kr_19-20": "float64",
        "kr_20-21": "float64",
        "kr_21-22": "float64",
        "kr_22-23": "float64",
        "kr_23-24": "float64",
        "krz_17-18": "float64",
        "krz_18-19": "float64",
        "krz_19-20": "float64",
        "krz_20-21": "

df.shape

### 8. Using the raster and vector items created and generating the Catalog 

In [10]:

catalog = pystac.Catalog(
    id=constants.id_main,
    title=constants.title_main,
    description=constants.description_main
)
catalog.add_item(create_raster_item(filename=raster_filename))
catalog.add_item(create_vector_item(vector_filename=vector_filename))
catalog.set_self_href(os.path.join(gobindpur_dir, "catalog.json"))

corestack_catalog = pystac.Catalog(
    id="corestack",
    title="CorestackCatalogs",
    description="Root catalog containing all subcatalogs"
)
corestack_catalog.add_child(catalog)
corestack_catalog.set_self_href(os.path.join(corestack_dir, "catalog.json"))
corestack_catalog.normalize_and_save(corestack_dir, catalog_type=pystac.CatalogType.SELF_CONTAINED)

print(f" Root STAC Catalog created at: {os.path.join(corestack_dir, 'catalog.json')}")


saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif
2023-07-01 00:00:00
2024-06-30 00:00:00
[{'value': 0, 'label': 'clear', 'alpha': '0', 'color': '#000000'}, {'value': 1, 'label': 'built up', 'alpha': '255', 'color': '#ff0000'}, {'value': 2, 'label': 'kharif water', 'alpha': '255', 'color': '#74ccf4'}, {'value': 3, 'label': 'kharif and rabi water', 'alpha': '255', 'color': '#1ca3ec'}, {'value': 4, 'label': 'kharif and rabi and zaid water', 'alpha': '255', 'color': '#0f5e9c'}, {'value': 5, 'label': 'croplands', 'alpha': '255', 'color': '#f1c232'}, {'value': 6, 'label': 'Tree/Forests', 'alpha': '255', 'color': '#38761d'}, {'value': 7, 'label': 'barren lands', 'alpha': '255', 'color': '#a9a9a9'}, {'value': 8, 'label': 'Single Kharif Cropping', 'alpha': '255', 'color': '#bad93e'}, {'value': 9, 'label': 'Single Non-Kharif Cropping', 'alpha': '255', 'color': '#f59d22'}, {'value': 10, 'label': 'Double Cropping', 'alpha': '255', 'color': '#ff9371'}, {'value': 11, 'label': 'Tri